In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime
import random

# Base de dados

In [2]:
df = pd.read_excel('../Base120.xlsx')

In [3]:
def filtra_por_solucao(df, vet_solucao):
    '''Recebe um vetor solução e retorna o df filtrado para essa solução'''
    stand_prescription_map = {i + 1: vet_solucao[i] for i in range(len(vet_solucao))}
    filtered_df = df[df.apply(lambda row: row['prescrição'] == stand_prescription_map.get(row['talhao']), axis=1)]
    return filtered_df

def calcula_volume(result):
	"""
	Calculate the volume by summing specific columns of a DataFrame.
	This function takes a DataFrame `result` and returns the sum of the values
	in columns 3 to 18 (inclusive).
	Parameters:
	result (pandas.DataFrame): The input DataFrame containing the data.
	Returns:
	pandas.Series: A Series containing the sum of the specified columns.
	"""
	
    
	return result.iloc[:, 3:19].sum()

def calcula_penalidade(volume_anual):
    '''retorna penalidade por volume excedente/faltante (R$500)'''
    upper = []
    lower = []
    for v_ano in volume_anual:
        if v_ano < 140000:
            lower.append(np.abs(v_ano - 140000))
        elif v_ano > 160000:
            upper.append(np.abs(v_ano - 160000))
    return np.sum(upper + lower)*500

def calcula_vpl_aux(result, volume_ano):
    '''Calcula o VPL penalizado (função auxiliar)'''
    return np.sum(result['VPL']) - calcula_penalidade(volume_ano)

def plota_volume(df, vet_sol):
    '''Plota o gráfico do volume nos anos'''
    volume_ano = filtra_por_solucao(df, vet_sol).iloc[:, 3:19].sum()
    (volume_ano/1000).plot(kind='bar')
    plt.hlines(140, xmin=-1, xmax=16, color='red', alpha=.6)
    plt.hlines(160, xmin=-1, xmax=16, color='red', alpha=.6)
    plt.title("Volume de madeira por ano")
    plt.xlabel("Ano de Produção")
    plt.ylabel("Volume (1000 m$^3$)")

def calcula_volume_aux(df, vet_sol):
	"""
	Calculate the auxiliary volume based on the given solution vector.
	This function filters the DataFrame `df` using the solution vector `vet_sol`
	and then sums the values in columns 3 to 18 (inclusive) of the filtered DataFrame.
	Parameters:
	df (pandas.DataFrame): The input DataFrame containing the data.
	vet_sol (list or array-like): The solution vector used to filter the DataFrame.
	Returns:
	pandas.Series: A Series containing the sum of the values in columns 3 to 18 of the filtered DataFrame.
	"""
    
	return filtra_por_solucao(df, vet_sol).iloc[:, 3:19].sum()

def calcula_vpl_total(df, vet_solucao):
    '''Calcula o VPL total penalizado'''
    result = filtra_por_solucao(df, vet_solucao)
    volume_ano = calcula_volume(result)
    return calcula_vpl_aux(result, volume_ano)

def encontra_pior_ano(df, sol):
    volume_ano = calcula_volume_aux(df, sol)
    return np.argmax(np.abs(volume_ano-160000)) + 1

def numpy_to_python(obj):
    '''Converte np.int e np.float para int e float dos dados do dicionário'''
    if isinstance(obj, (np.integer, np.floating)):
        return obj.item()
    
def python_to_numpy(dicionario):
    '''Converte int e float para np.int e np.float dos dados do dicionário'''
    for chave, subdict in dicionario.items():
        for subchave, valor in subdict.items():
            if isinstance(valor, int):
                subdict[subchave] = np.int64(valor)
            elif isinstance(valor, float):
                subdict[subchave] = np.float64(valor)
                
    dicionario = {np.int64(key): valor for key, valor in dicionario.items()}
    return dicionario

---

## Alteração na base de dados

In [4]:
df_2 = df.drop_duplicates(subset=['talhao'] + df.columns[3:].tolist(), keep='first')
df_2.to_csv('../base_reduzida.csv', index=False)

---

# Algoritmo Genético

## Soluções importadas


In [5]:
with open("../solucoes_alpha.json", "r") as arquivo: # soluções geradas por heurística construtiva
    solucoes_alpha = json.load(arquivo)
solucoes_alpha = [np.array(sol) for sol in solucoes_alpha]

In [6]:
class solucao:
    def __init__(self, sol):
        self.sol = sol
        self.vpl = calcula_vpl_total(df_2, self.sol)

In [28]:
def gera_populacao_inicial(n_pop, n_talhoes=120, perc_aleatoria=0.8) -> list[solucao]:
    populacao = [solucao(sol) for sol in np.random.randint(1, 121, size=(round(n_pop*perc_aleatoria), n_talhoes))]  # pop aleatória
    melhor_sol = solucoes_alpha[2]
    
    for _ in range(n_pop-round(n_pop*perc_aleatoria)): # quantidade soluções semi boas
        sol = np.copy(melhor_sol)
        random.shuffle(sol)
        populacao.append(solucao(sol))
    return populacao

def selecao(populacao, metodo='torneio') -> np.ndarray[solucao]:
    pares = np.array([[],[]])
    tam_torneio = 0.15
    
    for _ in range(round(len(populacao) / 2)): # quantidade de pares
        populacao_aux = populacao.copy()
        pop_torneio1 = random.sample(populacao_aux, round(len(populacao_aux) * tam_torneio))
        p1 = max(pop_torneio1, key=lambda aux: calcula_vpl_total(df_2, aux.sol))
        populacao_aux.remove(p1)
        
        pop_torneio2 = random.sample(populacao_aux, round(len(populacao_aux) * tam_torneio))
        p2 = max(pop_torneio2, key=lambda aux: calcula_vpl_total(df_2, aux.sol))
        pares = np.append(pares, [[p1], [p2]])
        
    return pares.reshape(round(len(populacao)/2), 2)

def recombinacao(pares, prob_mutacao) -> list[solucao]:
    filhos = []
    pontos_crossover = [round(len(pares[0][0].sol) * random.random()) for _ in range(2)]
    pontos_crossover.sort()
    crossover1 = pontos_crossover[0]
    crossover2 = pontos_crossover[1]
    
    for pai1, pai2 in pares:
        filho1 = solucao(np.concatenate((pai1.sol[:crossover1], 
                                         pai2.sol[crossover1:crossover2], 
                                         pai1.sol[crossover2:])))
        
        filho2 = solucao(np.concatenate((pai2.sol[:crossover1], 
                                         pai1.sol[crossover1:crossover2], 
                                         pai2.sol[crossover2:])))
        if random.random() < prob_mutacao:
            mutacao(filho1)
        if random.random() < prob_mutacao:
            mutacao(filho2)
        filhos.append(filho1)
        filhos.append(filho2)
    return populacao + filhos
    
def mutacao(filho) -> None:
    talhao_sorteado = random.randrange(1, 121)
    presc_sorteada = random.choice(df_2[df_2['talhao'] == (talhao_sorteado)]['prescrição'].unique())
    filho.sol[talhao_sorteado-1] = presc_sorteada

populacao = gera_populacao_inicial(20)
pares = selecao(populacao)
pop_recombinada = recombinacao(pares, prob_mutacao=0.05)

In [29]:
for sol in pop_recombinada:
    print(calcula_vpl_total(df_2, sol.sol))

-920799449
-903459007
-905676701
-962615648
-1001331617
-946217028
-841494793
-885993579
-928972420
-848569929
-893414740
-827887620
-894392670
-896868938
-974687139
-750561522
-157548492
-114346065
-230432400
-199678468
-217141169
-684901689
-121403144
-152398413
-896462703
-903606668
-155926218
-173470815
-325426538
-754740879
-253725243
-166025549
-751303197
-840753118
-581786172
-296574842
-639017231
-316849881
-160677625
-253725243
